# Create the common working RGI list and analyse amount of errors per region ... 
- uses the RGI batch files `/home/www/lschuster/provide/gfdl-esm2m_oversh_stab_uni_bern/runs/output/RGI{rgi_reg_s}/run_hydro_w5e5_gcm_merged_from_2000_gfdl-esm2...` as input 
- creates `../data/working_rgis_for_oversh_stab_scenario_bc_1980_2019.csv`
- creates `../data/working_rgis_for_oversh_stab_scenario_bc_2000_2019.csv`
    - uses another bias correction period
- creates `../data/random_climate_run_10000years_working_rgis_for_oversh_stab_scenarios_1980_2019.csv`


In [13]:
from oggm import cfg, workflow, utils, shop
import pandas as pd
import geopandas as gpd
import os, glob
import numpy as np
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

sns.set_style("whitegrid")

cfg.initialize()

matplotlib.rcParams['figure.figsize'] = (14, 8)
frgi = utils.file_downloader('https://cluster.klima.uni-bremen.de/~oggm/rgi/rgi62_stats.h5')
#frgi = '/home/users/lschuster/glacierMIP/rgi62_stats.h5'
odf = pd.read_hdf(frgi, index_col=0)
rgi_reg_s = '05'
rgi_ids = gpd.read_file(utils.get_rgi_region_file(rgi_reg_s, version='62'))
if rgi_reg_s == '05':
    rgidf = rgi_ids.loc[(rgi_ids['Connect'] == 0) | (rgi_ids['Connect'] ==1)]
odf = odf.loc[(odf['Connect'] == 0) | (odf['Connect'] ==1)]

2024-08-02 21:15:05: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2024-08-02 21:15:05: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2024-08-02 21:15:05: oggm.cfg: Multiprocessing: using all available processors (N=32)


In [14]:
scenarios = ['stab_T12','stab_T15','oversh_T20OS15','oversh_T25OS15',
                     'oversh_T30OS15','stab_T20','stab_T25','stab_T30']

**Preprocessing errors** 
- failing glaciers and area before starting the actual future projections

In [15]:
rgi_reg_l = []
for rgi_reg in np.arange(1,20,1):
    if rgi_reg <10:
        rgi_reg = f'0{rgi_reg}'
    else:
        rgi_reg = f'{rgi_reg}'
    rgi_reg_l.append(rgi_reg)
rgi_reg_l.append('all')


columns = ['rgi_reg', 'option','version','border', 'perc_area_miss', 'perc_glac_miss'] #+ opts + opts_w
pd_stats_l = []
working_rgis_l = []
option = 'W5E5_spinup'
border = 'b_160'
pd_stats = pd.DataFrame(columns=columns,
                index = rgi_reg_l)
path = '/home/www/oggm/gdirs/oggm_v1.6/'
add = f'L3-L5_files/2023.3/elev_bands/{option}/RGI62/{border}/L5/summary/'
stats_l = []
for rgi_reg in rgi_reg_l[:-1]:
    stats_l.append(pd.read_csv(path+add+f'glacier_statistics_{rgi_reg}.csv', low_memory=False))
stats = pd.concat(stats_l)  

stats_w = stats.loc[stats.error_msg.isna()]
working_rgis_l.append(stats_w.rgi_id.values)
perc_area_miss = 100-100*stats_w.rgi_area_km2.sum()/stats.rgi_area_km2.sum()
### 
print(f'global missing area: {perc_area_miss.round(2)}%')

global missing area: 0.1%


In [16]:
## largest amount of failing 
for rgi_reg in np.arange(1,19.1,1):
    area_w = stats_w.loc[stats_w.rgi_region == rgi_reg].rgi_area_km2.sum()/ stats.loc[stats.rgi_region == rgi_reg].rgi_area_km2.sum()
    print(f'RGI{rgi_reg}: working area = {(area_w*100).round(2)}%') 

RGI1.0: working area = 99.99%
RGI2.0: working area = 99.96%
RGI3.0: working area = 99.96%
RGI4.0: working area = 99.99%
RGI5.0: working area = 100.0%
RGI6.0: working area = 100.0%
RGI7.0: working area = 100.0%
RGI8.0: working area = 99.96%
RGI9.0: working area = 99.96%
RGI10.0: working area = 96.34%
RGI11.0: working area = 99.9%
RGI12.0: working area = 88.06%
RGI13.0: working area = 99.86%
RGI14.0: working area = 99.98%
RGI15.0: working area = 99.95%
RGI16.0: working area = 99.96%
RGI17.0: working area = 99.89%
RGI18.0: working area = 99.93%
RGI19.0: working area = 99.81%


**Region 12 has most failing area, but is also very small** (see stats below)

In [17]:
# very small area compared to global glacier area
stats.loc[stats.rgi_region == 12].rgi_area_km2.sum() / stats.rgi_area_km2.sum()

0.0018519486456116474

## Get a list with the amount of failing glaciers and area for the different scenarios

In [18]:
pd_geodetic = utils.get_geodetic_mb_dataframe()[utils.get_geodetic_mb_dataframe().period=='2000-01-01_2020-01-01']
total_area = pd_geodetic.area.sum()
total_counts = len(pd_geodetic)
# pd_geodetic.area
pd_working = pd.DataFrame(index = pd_geodetic.index,
                          columns=scenarios)
# we will set those that are not running afterwards to np.NaN value
pd_working.loc[pd_geodetic.index] = True
pd_working['area'] = pd_geodetic.area
pd_working['all_running_rgis'] = np.NaN
pd_working['rgi_reg'] = pd_geodetic.reg

In [6]:
for bc in ['_bc_1980_2019','_bc_2000_2019']:
    print(bc)
    for rgi_reg in pd_working.rgi_reg.unique():
        print(rgi_reg)
        if rgi_reg < 10:
            rgi_reg_s = f'0{rgi_reg}'
        else:
            rgi_reg_s = f'{rgi_reg}'
        dpath = f'/home/www/lschuster/provide/gfdl-esm2m_oversh_stab_uni_bern/runs/output/RGI{rgi_reg_s}'
        # amount of glaciers int that rgi region
        rgi_reg_glaciers = pd_working.loc[pd_working.rgi_reg==int(rgi_reg)].index
        for scenario in scenarios:    
            with xr.open_mfdataset(f'{dpath}/run_hydro_w5e5_gcm_merged_from_2000_gfdl-esm2m_{scenario}_endyr_2500{bc}_rgi{rgi_reg_s}*.nc') as ds:
                ds = ds.volume.sel(time=2500).load()
                # make sure that all glaciers have been running
                # print(len(ds.rgi_id.values), len(rgi_reg_glaciers))
                try:
                    assert len(ds.rgi_id.values) == len(rgi_reg_glaciers)
                except:
                    print('missing',len(ds.rgi_id.values), len(rgi_reg_glaciers))
                rgis_error = list(set(rgi_reg_glaciers) - set(ds.dropna(dim='rgi_id').rgi_id.values))
                pd_working.loc[rgis_error, scenario] = np.NaN

    all_running_rgis = pd_working[scenarios].dropna().index
    pd_working.loc[all_running_rgis, 'all_running_rgis'] = True
    all_running_rel = len(all_running_rgis)*100/len(pd_geodetic)
    all_running_rel_area_geod = pd_working.loc[all_running_rgis].area.sum()*100/pd_working.area.sum()
    all_running_rel_area_rgi = odf.loc[all_running_rgis].Area.sum()*100/odf.Area.sum()

    print(f'Amount of glaciers that run over the entire time period: {len(all_running_rgis)}')
    print(f'Relative percentage of glacier amount where all scenarios could run over the entire time period: {all_running_rel:0.2f}%')
    print(f'Relative percentage of glacier area where all scenarios could run over the entire time period: via geod {all_running_rel_area_geod:0.2f}%, via rgi area {all_running_rel_area_rgi:0.2f}%')
    #pd_rel_error_area_L5.to_csv('rel_error_area_statistics_for_oversh_stab_scenarios.csv')
    pd_working.to_csv(f'../data/working_rgis_for_oversh_stab_scenarios{bc}.csv')
    print('\n')

_bc_1980_2019
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Amount of glaciers that run over the entire time period: 212759
Relative percentage of glacier amount where all scenarios could run over the entire time period: 98.71%
Relative percentage of glacier area where all scenarios could run over the entire time period: via geod 99.58%, via rgi area 99.57%


_bc_2000_2019
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Amount of glaciers that run over the entire time period: 211915
Relative percentage of glacier amount where all scenarios could run over the entire time period: 98.31%
Relative percentage of glacier area where all scenarios could run over the entire time period: via geod 99.20%, via rgi area 99.20%




##### Do the same for every RGI region:

In [10]:
pd_working = pd.read_csv(f'../data/working_rgis_for_oversh_stab_scenarios_bc_1980_2019.csv', index_col=[0])
 
for rgi_reg in pd_working.rgi_reg.unique():
    if rgi_reg < 10:
        rgi_reg_s = f'0{rgi_reg}'
    else:
        rgi_reg_s = str(rgi_reg)
    pd_working_sel = pd_working.loc[pd_working.rgi_reg==int(rgi_reg)]
    all_running_rgis_reg = pd_working_sel[scenarios].dropna().index
    all_running_rel_reg = len(all_running_rgis_reg)*100/len(pd_working_sel)
    all_running_rel_area_reg = pd_working.loc[all_running_rgis_reg].area.sum()*100/pd_working_sel.area.sum()
    
    odf_reg = odf.loc[odf.O1Region==rgi_reg_s]
    all_running_rel_area_reg_rgi = odf_reg.loc[all_running_rgis_reg].Area.sum()*100/odf_reg.Area.sum()

    print(f'RGI{rgi_reg_s}')
    print(f'Amount of glaciers that run over the entire time period: {len(all_running_rgis_reg)}')
    print(f'Relative percentage of glacier amount where all scenarios could run over the entire time period: {all_running_rel_reg:0.2f}%')
    print(f'Relative percentage of glacier area where all scenarios could run over the entire time period: via geod {all_running_rel_area_reg:0.2f}%, via area {all_running_rel_area_reg_rgi:0.2f}%')
    print(f'\n')

RGI01
Amount of glaciers that run over the entire time period: 27026
Relative percentage of glacier amount where all scenarios could run over the entire time period: 99.70%
Relative percentage of glacier area where all scenarios could run over the entire time period: via geod 99.98%, via area 99.98%


RGI02
Amount of glaciers that run over the entire time period: 18721
Relative percentage of glacier amount where all scenarios could run over the entire time period: 99.29%
Relative percentage of glacier area where all scenarios could run over the entire time period: via geod 99.96%, via area 99.96%


RGI03
Amount of glaciers that run over the entire time period: 4495
Relative percentage of glacier amount where all scenarios could run over the entire time period: 98.66%
Relative percentage of glacier area where all scenarios could run over the entire time period: via geod 99.97%, via area 99.97%


RGI04
Amount of glaciers that run over the entire time period: 7348
Relative percentage of g

### Now get the common glaciers for the random climate runs:

- less glaciers are running, specifically in RGI 03,07, 09, 19

In [19]:
pd_geodetic = utils.get_geodetic_mb_dataframe()[utils.get_geodetic_mb_dataframe().period=='2000-01-01_2020-01-01']
total_area = pd_geodetic.area.sum()
total_counts = len(pd_geodetic)
# pd_geodetic.area
pd_working = pd.DataFrame(index = pd_geodetic.index,
                          columns=['oversh_T30OS15','stab_T15'])
# we will set those that are not running afterwards to np.NaN value
pd_working.loc[pd_geodetic.index] = True
pd_working['area'] = pd_geodetic.area
pd_working['all_running_rgis'] = np.NaN
pd_working['rgi_reg'] = pd_geodetic.reg

for bc in ['_bc_1980_2019']:
    print(bc)
    for rgi_reg in pd_working.rgi_reg.unique():
        print(rgi_reg)
        if rgi_reg < 10:
            rgi_reg_s = f'0{rgi_reg}'
        else:
            rgi_reg_s = f'{rgi_reg}'
        dpath = f'/home/www/lschuster/provide/gfdl-esm2m_oversh_stab_uni_bern/runs/output/RGI{rgi_reg_s}'
        # amount of glaciers int that rgi region
        rgi_reg_glaciers = pd_working.loc[pd_working.rgi_reg==int(rgi_reg)].index
        for scenario in ['oversh_T30OS15','stab_T15']: # ,'zero']:    <-- we don't use the scenario where we start at zero... 
            with xr.open_mfdataset(f'{dpath}/run_random_climate_from2500_using2400_2500_gfdl-esm2m_stab_T15_initial_{scenario}{bc}_rgi{rgi_reg_s}_*.nc') as ds:
                ds = ds.volume.isel(time=slice(0,-1)).load()
                # check if we have only run for less than 11000 years
                assert ds.time.max() <11000
                # make sure that all glaciers have been running
                assert len(ds.rgi_id.values) == len(rgi_reg_glaciers)
                rgis_error = list(set(rgi_reg_glaciers) - set(ds.dropna(dim='rgi_id').rgi_id.values))
                pd_working.loc[rgis_error, scenario] = np.NaN

    all_running_rgis = pd_working[['oversh_T30OS15','stab_T15']].dropna().index
    pd_working.loc[all_running_rgis, 'all_running_rgis'] = True
    all_running_rel = len(all_running_rgis)*100/len(pd_geodetic)
    all_running_rel_area = pd_working.loc[all_running_rgis].area.sum()*100/pd_working.area.sum()

    print(f'Amount of glaciers that run over the entire time period: {len(all_running_rgis)}')
    print(f'Relative percentage of glacier amount where all scenarios could run over the entire time period: {all_running_rel:0.2f}%')
    print(f'Relative percentage of glacier area where all scenarios could run over the entire time period: {all_running_rel_area:0.2f}%')
    #pd_rel_error_area_L5.to_csv('rel_error_area_statistics_for_oversh_stab_scenarios.csv')
    pd_working.to_csv(f'../data/random_climate_run_10000years_working_rgis_for_oversh_stab_scenarios{bc}.csv')
    print('\n')

_bc_1980_2019
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Amount of glaciers that run over the entire time period: 211908
Relative percentage of glacier amount where all scenarios could run over the entire time period: 98.31%
Relative percentage of glacier area where all scenarios could run over the entire time period: 95.96%




In [20]:
bc = '_bc_1980_2019'
pd_working = pd.read_csv(f'../data/random_climate_run_10000years_working_rgis_for_oversh_stab_scenarios{bc}.csv', index_col=[0])

for rgi_reg in pd_working.rgi_reg.unique():
    if rgi_reg < 10:
        rgi_reg_s = f'0{rgi_reg}'
    else:
        rgi_reg_s = str(rgi_reg)
    pd_working_sel = pd_working.loc[pd_working.rgi_reg==int(rgi_reg)]
    all_running_rgis_reg = pd_working_sel[['oversh_T30OS15','stab_T15']].dropna().index
    all_running_rel_reg = len(all_running_rgis_reg)*100/len(pd_working_sel)
    #all_running_rel_area_reg = pd_working.loc[all_running_rgis_reg].area.sum()*100/pd_working_sel.area.sum()
    odf_reg = odf.loc[odf.O1Region==rgi_reg_s]
    all_running_rel_area_reg_rgi = odf_reg.loc[all_running_rgis_reg].Area.sum()*100/odf_reg.Area.sum()
    print(f'RGI{rgi_reg_s}')
    print(f'Amount of glaciers that run over the entire time period: {len(all_running_rgis_reg)}')
    print(f'Relative percentage of glacier amount where all scenarios could run over the entire time period: {all_running_rel_reg:0.2f}%')
    print(f'Relative percentage of glacier area where all scenarios could run over the entire time period: {all_running_rel_area_reg_rgi:0.2f}%')
    print(f'\n')

RGI01
Amount of glaciers that run over the entire time period: 27010
Relative percentage of glacier amount where all scenarios could run over the entire time period: 99.64%
Relative percentage of glacier area where all scenarios could run over the entire time period: 99.80%


RGI02
Amount of glaciers that run over the entire time period: 18693
Relative percentage of glacier amount where all scenarios could run over the entire time period: 99.14%
Relative percentage of glacier area where all scenarios could run over the entire time period: 99.95%


RGI03
Amount of glaciers that run over the entire time period: 4359
Relative percentage of glacier amount where all scenarios could run over the entire time period: 95.68%
Relative percentage of glacier area where all scenarios could run over the entire time period: 94.74%


RGI04
Amount of glaciers that run over the entire time period: 7196
Relative percentage of glacier amount where all scenarios could run over the entire time period: 97.05